In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [2]:
data = pd.read_csv('steady_state_CSTR_data2_last_ouput as true data only.csv').values

In [3]:
data = data[:, 1:]
F_by_V = data[:, 0]
CAf = data[:, 1]
CBf = data[:, 2]
CCf = data[:, 3]
CDf = data[:, 4]
CA = data[:, 5]
CB = data[:, 6]
CC = data[:, 7]
CD = data[:, 8]
tt = data.shape[0]  # Total data
tn = int(0.6 * tt)  # Total Number of Training Data
tv = tt - tn  # Total Number of Validation Data
nt = 9  # Total number of variables
ni = 5  # Number of neurons in Input Layer
no = 4  # Number of neurons in Output Layer

In [4]:
Imat_not_scaled = data[:, 0:5].T
Imat_t_not_scaled=Imat_not_scaled[: , :int(0.6 * Imat_not_scaled.shape[1])]

CAf_t_not_scaled = Imat_t_not_scaled[1, :]
CBf_t_not_scaled = Imat_t_not_scaled[2, :]
CCf_t_not_scaled = Imat_t_not_scaled[3, :]
CDf_t_not_scaled = Imat_t_not_scaled[4, :]


Imat_not_scaled = data[:, 0:5].T
Imat_v_not_scaled = Imat_not_scaled[:, -int(0.4 * Imat_not_scaled.shape[1]):]

CAf_v_not_scaled = Imat_v_not_scaled[1, :]
CBf_v_not_scaled = Imat_v_not_scaled[2, :]
CCf_v_not_scaled = Imat_v_not_scaled[3, :]
CDf_v_not_scaled = Imat_v_not_scaled[4, :]

In [5]:
C_input_training = {
    'CA_input_training': CAf_t_not_scaled,
    'CB_input_training': CBf_t_not_scaled,
    'CC_input_training': CCf_t_not_scaled,
    'CD_input_training': CDf_t_not_scaled
}
file_name_input = 'C_input_training_with true data output.csv'
df_input = pd.DataFrame(C_input_training)
df_input.to_csv(file_name_input, index=False)

In [6]:
C_input_validation = {
    'CA_input_validation': CAf_v_not_scaled,
    'CB_input_validation': CBf_v_not_scaled,
    'CC_input_validation': CCf_v_not_scaled,
    'CD_input_validation': CDf_v_not_scaled
}
file_name_input2 = 'C_input_validation_with true data output.csv'
df_input2 = pd.DataFrame(C_input_validation)
df_input2.to_csv(file_name_input2, index=False)

In [7]:
norm_mat = np.zeros((tt, nt))
delta = np.zeros(nt)
for i in range(nt):
    delta[i] = np.max(data[:, i]) - np.min(data[:, i])
    norm_mat[:, i] = (data[:, i] - np.min(data[:, i])) / delta[i]  # Max-Min Normalization

In [8]:
dsr = norm_mat[:, 5:9].T  # Target array
Imat = norm_mat[:, 0:5].T
dsr_t=dsr[: , :int(0.6 * dsr.shape[1])]
Imat_t=Imat[: , :int(0.6 * Imat.shape[1])]
dsr_v = dsr[:, -int(0.4 * dsr.shape[1]):]
Imat_v = Imat[:, (-int(0.4 * Imat.shape[1])):]

In [9]:
Q = np.array([
    [5, 5, 5, 10],
    [10, 10, 10, 20],
    [2, 2, 2, 4]])

CAf_t = Imat_t[1, :]
CBf_t = Imat_t[2, :]
CCf_t = Imat_t[3, :]
CDf_t = Imat_t[4, :]
K = np.array([
    5*CAf_t + 5*CBf_t + 5*CCf_t + 10*CDf_t,
    10*CAf_t + 10*CBf_t + 10*CCf_t + 20*CDf_t,
    2*CAf_t + 2*CBf_t + 2*CCf_t + 4*CDf_t ])

In [10]:
# Number of centers of RBFs
tc = 50

# # Getting coordinates of the centres
# distances = np.zeros(len(Imat_t[1]))
# mean1=np.mean(Imat_t[:,0])
# mean2=np.mean(Imat_t[:,1])
# mean3=np.mean(Imat_t[:,2])
# mean4=np.mean(Imat_t[:,3])
# mean5=np.mean(Imat_t[:,4])
# mean = np.array([mean1, mean2, mean3, mean4, mean5])
# for i in range(len(Imat_t)):
#     for j in range(ni):
#         distances[i]+=Imat_t[j][i]-mean[j]
# indices = np.argsort(distances)   # Sorting the distances and get corresponding indices
# # for l in range(tc):
# #     centermat=Imat_t[indices[l]]
# centermat = Imat_t[:,indices[:tc]]   # Select 'tc'-nearest neighbors as the centres

# Getting coordinates of the centres-Probabilistic Centre Selection
centermat = np.zeros((ni, tc))
for i in range(ni):
    mean_i = np.mean(Imat_t[i, :])
    sigma_i = 1
    x = np.linspace(mean_i-3*sigma_i, mean_i+3*sigma_i, Imat_t.shape[1])
    curve = 1/np.sqrt(2*np.pi*sigma_i**2)*np.exp(-(x-mean_i)**2/(2*sigma_i**2))
    random_distributed_indices = np.random.choice(len(curve), size=tc, replace=True)
    centermat[i, :] = Imat_t[i][random_distributed_indices]


#Getting width
#sigma = 2
kmeans = KMeans(n_clusters=tc)
kmeans.fit(Imat_t.T)
centers = kmeans.cluster_centers_.T
sigma = np.mean(np.linalg.norm(centermat - centers, axis=0))

#Getting Weights by Least Square method
D = np.zeros((tc, tn))
for i in range(tc):
    for j in range(tn):
        D[i, j] = np.linalg.norm(centermat[:, i] - Imat_t[:, j])
PhiofD = 1 / np.sqrt(2 * np.pi * sigma**2) * np.exp(-D**2 / (2 * sigma**2))  
pinv_phi = np.linalg.pinv(PhiofD)
pinv_Q = np.linalg.pinv(Q)
#z = np.dot(np.dot(pinv_Q, K),pinv_phi).T
z = np.dot(pinv_phi.T, (np.dot(pinv_Q, K)).T)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


#Training

In [11]:
D = np.zeros((tc, tn))
for i in range(tc):
    for j in range(tn):
        D[i, j] = np.linalg.norm(centermat[:, i] - Imat_t[:, j])
yHL = 1 / np.sqrt(2 * np.pi * sigma**2) * np.exp(-D**2 / (2 * sigma**2))
ynn_tv = np.dot(yHL.T, z)  # Prediction of Training Data
dsr_t_v = dsr_t.T*delta[ni:] + np.min(data[:, ni:], axis=0)
ynn_t_v = ynn_tv*delta[ni:] + np.min(data[:, ni:], axis=0)

In [12]:
CA_t_p = ynn_t_v[:, 0]
CB_t_p = ynn_t_v[:, 1]
CC_t_p = ynn_t_v[:, 2]
CD_t_p = ynn_t_v[:, 3]

In [13]:
C_output_MCNN_training = {
    'CA_output_MCNN_training': CA_t_p,
    'CB_output_MCNN_training': CB_t_p,
    'CC_output_MCNN_training': CC_t_p,
    'CD_output_MCNN_training': CD_t_p
}
# file_name1 = 'C_output_MCNN_training_with true data output.csv'
# df1 = pd.DataFrame(C_output_MCNN_training)
# df1.to_csv(file_name1, index=False)

#Validation

In [14]:
Dv = np.zeros((tc, tv))
for i in range(tc):
    for j in range(tv):
        Dv[i, j] = np.linalg.norm(centermat[:, i] - Imat_v[:, j])
yHL = 1 / np.sqrt(2 * np.pi * sigma**2) * np.exp(-Dv**2 / (2 * sigma**2))
ynn_v = np.dot(yHL.T, z)
dsr_v_p = dsr_v.T * delta[ni:] + np.min(data[:, ni:], axis=0)
ynn_v_p = ynn_v * delta[ni:] + np.min(data[:, ni:], axis=0)

In [15]:
CA_v_p = ynn_v_p[:, 0]
CB_v_p = ynn_v_p[:, 1]
CC_v_p = ynn_v_p[:, 2]
CD_v_p = ynn_v_p[:, 3]

In [16]:
C_output_MCNN_validation = {
    'CA_output_MCNN_validation': CA_v_p,
    'CB_output_MCNN_validation': CB_v_p,
    'CC_output_MCNN_validation': CC_v_p,
    'CD_output_MCNN_validation': CD_v_p
}
# file_name2 = 'C_output_MCNN_validation_with true data output.csv'
# df2 = pd.DataFrame(C_output_MCNN_validation)
# df2.to_csv(file_name2, index=False)